In [1]:
import pandas as pd 
import ast 

In [2]:
test_wallet_timeseries = pd.read_csv('../data/test_acc_timeseries.csv')

In [3]:
test_wallet_timeseries

total_tx = len(test_wallet_timeseries['signature'].unique())
total_tx

493

In [4]:
test_wallet_timeseries['direction'].unique()

array(['received', 'sent', nan], dtype=object)

In [5]:
received_tx = test_wallet_timeseries[test_wallet_timeseries['direction']=='received']
sent_tx = test_wallet_timeseries[test_wallet_timeseries['direction']=='sent']

In [6]:
sent_tx['net_token_flows'].iloc[0]


"{'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': -128.319372}"

In [7]:
sent_tx['net_token_flows'].iloc[1]


"{'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB': -129.499459, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 128.319372}"

In [8]:
sent_tx['net_token_flows'].iloc[2]


"{'SOL': -0.005, 'So11111111111111111111111111111111111111112': -0.005, 'J1toso1uCk3RLmjorhTtrVwY9HJ7X8V9yYac6Y7kGCPn': 0.004164116}"

In [9]:
sent_tx['net_token_flows'].dropna().head(5).tolist()


["{'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': -128.319372}",
 "{'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB': -129.499459, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 128.319372}",
 "{'SOL': -0.005, 'So11111111111111111111111111111111111111112': -0.005, 'J1toso1uCk3RLmjorhTtrVwY9HJ7X8V9yYac6Y7kGCPn': 0.004164116}",
 "{'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': -493.070747}",
 "{'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB': -497.448706, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 493.070747}"]

In [10]:
sample = sent_tx['net_token_flows'].dropna().iloc[0]
print("Raw value:", sample)

Raw value: {'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': -128.319372}


In [11]:
if isinstance(sample, str):
    try:
        parsed = ast.literal_eval(sample)
        print("Parsed dict:", parsed)
    except Exception as e:
        print("Failed to parse:", e)

Parsed dict: {'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': -128.319372}


In [12]:
import ast

parsed_flows = sent_tx['net_token_flows'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
valid_dicts = [d for d in parsed_flows if isinstance(d, dict) and d]
print(f"✅ {len(valid_dicts)} non-empty dicts found")

if valid_dicts:
    print("Sample dict:", valid_dicts[0])

✅ 136 non-empty dicts found
Sample dict: {'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': -128.319372}


In [13]:
from collections import Counter
import ast

# Parse strings into dictionaries safely
parsed_flows = sent_tx['net_token_flows'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Sum all values grouped by token address
total_flows = sum((Counter(flow) for flow in parsed_flows if isinstance(flow, dict)), Counter())

# Sort by absolute value descending (optional)
sorted_flows = dict(sorted(total_flows.items(), key=lambda item: abs(item[1]), reverse=True))

sorted_flows


{'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v': 152.548166,
 '9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY': 38.261386,
 'bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1': 5.262743093000001,
 'So11111111111111111111111111111111111111112': 1.8685639759999997}

In [14]:
parsed_flows = sent_tx['net_token_flows'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
parsed_flows.head(5).tolist()


[{'SOL': -0.0021924,
  '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': -128.319372},
 {'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB': -129.499459,
  '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 128.319372},
 {'SOL': -0.005,
  'So11111111111111111111111111111111111111112': -0.005,
  'J1toso1uCk3RLmjorhTtrVwY9HJ7X8V9yYac6Y7kGCPn': 0.004164116},
 {'SOL': -0.0021924,
  '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': -493.070747},
 {'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB': -497.448706,
  '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': 493.070747}]

In [15]:
valid_flows = [d for d in parsed_flows if isinstance(d, dict)]
print(f"✅ Found {len(valid_flows)} dicts")
print("Sample:", valid_flows[0])


✅ Found 136 dicts
Sample: {'SOL': -0.0021924, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': -128.319372}


In [16]:
from collections import defaultdict

agg = defaultdict(float)

for d in valid_flows:
    for token, amount in d.items():
        agg[token] += amount

# Optionally sort by absolute value
sorted_flows = dict(sorted(agg.items(), key=lambda x: abs(x[1]), reverse=True))
print(sorted_flows)

{'WENWENvqqNya429ubCdR81ZmD69brwQaaBYY6p3LCpk': -1287304.0, 'Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB': -6688.240062000001, '2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv': -3490.0, '2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo': -2055.9510000000005, 'EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v': -1976.4328239999998, '9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY': -151.38026299999996, 'SOL': -31.675079503999992, 'So11111111111111111111111111111111111111112': -26.586492489000005, 'bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1': 5.262743093000001, 'J1toso1uCk3RLmjorhTtrVwY9HJ7X8V9yYac6Y7kGCPn': -1.1226087359999999, 'mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So': -0.170000021, '9uvxPXzVYWe8qb7S4gUBqsvASmRMVWo5GSAcMmdZ1LQr': 0.0, '7dHbWXmci3dT8UFYWYZweBLXgycu7Y3iL6trKn1Y7ARj': 0.0}


In [17]:
pd.DataFrame([sorted_flows])

,WENWENvqqNya429ubCdR81ZmD69brwQaaBYY6p3LCpk,Es9vMFrzaCERmJfrF4H2FYD4KCoNkY11McCe8BenwNYB,2zMMhcVQEXDtdE6vsFS7S7D5oUodfJHE8vd1gnBouauv,2b1kV6DkPAnxd5ixfnxCpjxmKwqjjaYmCZfHsFu24GXo,EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,SOL,So11111111111111111111111111111111111111112,bSo13r4TkiE4KumL71LsHTPpL2euBYLFx6h9HP3piy1,J1toso1uCk3RLmjorhTtrVwY9HJ7X8V9yYac6Y7kGCPn,mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So,9uvxPXzVYWe8qb7S4gUBqsvASmRMVWo5GSAcMmdZ1LQr,7dHbWXmci3dT8UFYWYZweBLXgycu7Y3iL6trKn1Y7ARj
0,-1287304.0,-6688.240062,-3490.0,-2055.951,-1976.432824,-151.380263,-31.67508,-26.586492,5.262743,-1.122609,-0.17,0.0,0.0
